In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
from dataset import MNIST_Moving

In [3]:

train_set = MNIST_Moving(root='.data/mnist', train=True, download=True)
test_set = MNIST_Moving(root='.data/mnist', train=False, download=True)

batch_size = 100

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

Processing...
Done!


In [4]:
input= next(iter(train_loader))
print(input.shape)
vid_seq = input[0]
print(vid_seq.shape)

torch.Size([100, 20, 1, 64, 64])
torch.Size([20, 1, 64, 64])


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [19]:
class EncBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3,stride=1,padding=(1, 1))
        self.relu  = nn.ReLU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3,stride=1,padding=(1, 1))
    
    def forward(self, x):
        return self.conv2(self.relu(self.conv1(x)))

class Encoder(nn.Module):
    def __init__(self, chs):
        super().__init__()
        self.enc_blocks = nn.ModuleList([EncBlock(chs[i], chs[i+1]) for i in range(len(chs)-1)])
        self.pool       = nn.MaxPool2d(2)
    
    def forward(self, x):
        layers = []
        # print("Layer shape")
        # print(x.shape)
        for block in self.enc_blocks:
            x = block(x)
            # print(x.shape)
            
            x = self.pool(x)
            layers.append(x)
        return x

In [20]:
class DecBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.ConvTranspose2d(in_ch, out_ch, 3,stride=2,padding=(1, 1))
        self.conv2 = nn.ConvTranspose2d(out_ch, out_ch, 2,stride=1)
    
    def forward(self, x):
        return self.conv2((self.conv1(x)))

class Decoder(nn.Module):
    def __init__(self, chs):
        super().__init__()
        self.dec_blocks = nn.ModuleList([DecBlock(chs[i], chs[i+1]) for i in range(len(chs)-1)])
        
    
    def forward(self, x):
        layers = []
        # print("hello")
        # print(x.shape)
        for block in self.dec_blocks:
            x = block(x)
            # print(x.shape)
            layers.append(x)

        return x

In [21]:
class Embedded_Encoder(nn.Module):

    def __init__(self):
      super().__init__()
      self.firstEncoder= Encoder((1,16,64))
      self.firstEncoder.to(device)
      self.secondEncoder= Encoder((64,128))
      self.secondEncoder.to(device)
      self.thirdEncoder= Encoder((128,256))
      self.thirdEncoder.to(device)
    
    def forward(self, x):
      #list to store the outputs from each encoders
      encoder_outputs = []
      x=self.firstEncoder(x)
      encoder_outputs.append(x)
      x=self.secondEncoder(x)
      encoder_outputs.append(x)
      x=self.thirdEncoder(x)
      encoder_outputs.append(x)

      
      return encoder_outputs

The decoder implemented here is standalone. It does not take inputs from the conv lstm. Todo: Concatenate input from the convlstms with the ouputs from the corresponding encoders and pass them as inputs to each decoder. 

In [22]:
class Embedded_Decoder(nn.Module):
  
    def __init__(self):
      super().__init__()
      #The decoder is in reverse here 
      self.firstDecoder= Decoder((256,128))
      self.firstDecoder.to(device)
      self.secondDecoder= Decoder((128,64))
      self.secondDecoder.to(device)
      self.thirdDecoder= Decoder((64,16,1))
      self.thirdDecoder.to(device)
    
    def forward(self, x):
      #list to store the outputs from each decoders
      decoder_outputs = []
      x=self.firstDecoder(x)
      decoder_outputs.append(x)
      x=self.secondDecoder(x)
      decoder_outputs.append(x)
      x=self.thirdDecoder(x)
      decoder_outputs.append(x)

      
      return decoder_outputs

Using the input shape [20, 1, 64, 64] as conv2d has 4d weights. So to train we have to iterate through the batches and pass each video sequence to the encoder.

In [43]:
ModelEncoder=Embedded_Encoder()
print("The input is of the size",vid_seq.shape)
encoderLayerOutputs=ModelEncoder(vid_seq.float().cuda())
#These features or outputs from each encoder can be then passed on to the convlstm 
print("The outputs of each encoder in the embedded encoder:")
for ftr in encoderLayerOutputs: print(ftr.shape)

The input is of the size torch.Size([20, 1, 64, 64])
The outputs of each encoder in the embedded encoder:
torch.Size([20, 64, 16, 16])
torch.Size([20, 128, 8, 8])
torch.Size([20, 256, 4, 4])


In [44]:
ModelDecoder=Embedded_Decoder()
print("The input of the embedded decoder is the output from the last encoder in the embedded encoder",encoderLayerOutputs[-1].shape)
decoderLayerOutputs=ModelDecoder(encoderLayerOutputs[-1])
print("The outputs of each encoder in the embedded decoder:")
for ftr in decoderLayerOutputs: print(ftr.shape)

The input of the embedded decoder is the output from the last encoder in the embedded encoder torch.Size([20, 256, 4, 4])
The outputs of each encoder in the embedded decoder:
torch.Size([20, 128, 8, 8])
torch.Size([20, 64, 16, 16])
torch.Size([20, 1, 64, 64])


In [39]:
class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias,mode="zeros"):
        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias
        self.mode = mode

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)
        
       
        

    def forward(self, x, cur_state):
        h_cur, c_cur = cur_state
        x = x.to(device)
        h_cur = h_cur.to(device)
        # print(x.size())
        # print(h_cur.size())
        concat_input_hcur = torch.cat([x, h_cur], dim=1) 
        concat_input_hcur = concat_input_hcur.to(device)

        concat_input_hcur_conv = self.conv(concat_input_hcur)
        concat_input_hcur_conv = concat_input_hcur_conv.to(device)

        cc_input_gate, cc_forget_gate, cc_output_gate, cc_output = torch.split(concat_input_hcur_conv, self.hidden_dim, dim=1)
        # print("cci",cc_input_gate.shape)
        # print("ccf",cc_forget_gate.shape)
        # print("ccog",cc_output_gate.shape)
        # print("cco",cc_output.shape)
        # print("cccur",c_cur.shape)
        # print("wci",self.W_ci.shape)
        
        input_gate = torch.sigmoid(cc_input_gate +  c_cur)

        forget_gate = torch.sigmoid(cc_forget_gate +  c_cur)

        output = torch.tanh(cc_output)

        c_next = forget_gate * c_cur + input_gate * output

        output_gate = torch.sigmoid(cc_output_gate +  c_next)

        h_next = output * torch.tanh(c_next)

        return h_next, c_next

    def init_state(self, batch_size, image_size):
        height, width = image_size
        """ Initializing hidden and cell state """
        if(self.mode == "zeros"):
            h = torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device)
            c = torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device)
        elif(self.mode == "random"):
            h = torch.randn(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device)
            c = torch.randn(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device)
        elif(self.mode == "learned"):
            h = self.learned_h.repeat(batch_size, 1, height, width, device=self.conv.weight.device)
            c = self.learned_c.repeat(batch_size, 1, height, width, device=self.conv.weight.device)
        
        return h, c

        

class ConvLSTM(nn.Module):


  

    """ 
    Custom LSTM for images. Batches of images are fed to a Conv LSTM
    
    Args:
    -----
    input_dim: integer
        Number of channels of the input.
    hidden_dim: integer
        dimensionality of the states in the cell
    kernel_size: tuple
        size of the kernel for convolutions
    num_layers: integer
        number of stacked LSTMS
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
       
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers
        classifier_in_dim= (self.hidden_dim[0]*16*16)

        

        conv_lstms  = []
        # iterating over no of layers
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            conv_lstms.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.conv_lstms = nn.ModuleList(conv_lstms)

    def forward(self, x, hidden_state=None):
       

        x=x.unsqueeze(dim=1)
        b, _, _, h, w = x.size()

        if hidden_state is not None:
            raise NotImplementedError()
        else:
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        
        cur_layer_input = x
        output_list = []
        x_len = x.size(1)
        

        # iterating over no of layers
        for i in range(self.num_layers):

            h, c = hidden_state[i]
            each_layer_output = []
            # iterating over sequence length

            for t in range(x_len):
                h, c = self.conv_lstms[i](x=cur_layer_input[:, t, :, :, :],cur_state=[h, c])
                each_layer_output.append(h)

            stacked_layer_output = torch.stack(each_layer_output, dim=1)
            cur_layer_input = stacked_layer_output

            output_list.append(stacked_layer_output)

        if not self.return_all_layers:
            output_list = output_list[-1:]

        batch_shape = output_list[-1].shape[0]

        return torch.stack(each_layer_output), (h, c)

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.conv_lstms[i].init_state(batch_size, image_size))
        return init_states

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [40]:
conv_model_FirstEncoder= ConvLSTM(input_dim= 64, hidden_dim = 64, kernel_size = (5,5), num_layers= 1)
if torch.cuda.is_available():
    conv_model_FirstEncoder.to(device)

In [41]:
conv_model_SecondEncoder= ConvLSTM(input_dim= 128, hidden_dim = 128, kernel_size = (5,5), num_layers= 1)
if torch.cuda.is_available():
    conv_model_SecondEncoder.to(device)

In [42]:
conv_model_ThirdEncoder= ConvLSTM(input_dim= 256, hidden_dim = 256, kernel_size = (5,5), num_layers= 1)
if torch.cuda.is_available():
    conv_model_ThirdEncoder.to(device)